In [1]:
import copy

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim

import anndata as ad

In [2]:

data_dir = "/Users/apple/Desktop/QBS146_Bioinfo_I/final_project/dataset"
adata = ad.read_h5ad(data_dir+'/h5ad/2024_xenium_breast_tTMA1.h5ad')


# 

In [3]:
adata.shape

(342704, 541)

In [5]:
adata.obs.columns

Index(['x_centroid', 'y_centroid', 'transcript_counts', 'control_probe_counts',
       'control_codeword_counts', 'unassigned_codeword_counts',
       'deprecated_codeword_counts', 'total_counts', 'cell_area',
       'nucleus_area', 'x', 'y', 'core', 'tissue_type'],
      dtype='object')

In [9]:
adata.obsm.keys

<bound method Mapping.keys of AxisArrays with keys: spatial>